In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [2]:
event = spark.read.format("csv").option("header", "true").load("athlete_events.csv")
 
event.createOrReplaceTempView("AthleteEvents")
 
display(event)

DataFrame[ID: string, Name: string, Sex: string, Age: string, Height: string, Weight: string, Team: string, NOC: string, Games: string, Year: string, Season: string, City: string, Sport: string, Event: string, Medal: string]

In [4]:
#What is the most common Olympic event for male athletes?

from pyspark.sql.functions import count, desc

male_events = event.filter(event.Sex == "M")
male_events.groupBy("Event").agg(count("Event").alias("NumOccurrences")).orderBy(desc("NumOccurrences")).show(1)

+--------------------+--------------+
|               Event|NumOccurrences|
+--------------------+--------------+
|Football Men's Fo...|          5728|
+--------------------+--------------+
only showing top 1 row



In [32]:
#What is the most common Olympic event for female athletes?

from pyspark.sql.functions import count, desc

male_events = event.filter(event.Sex == "F")
male_events.groupBy("Event").agg(count("Event").alias("NumOccurrences")).orderBy(desc("NumOccurrences")).show(1)

+--------------------+--------------+
|               Event|NumOccurrences|
+--------------------+--------------+
|Volleyball Women'...|          1543|
+--------------------+--------------+
only showing top 1 row



In [7]:
#Which Olympic games had the highest number of participating athletes?
from pyspark.sql.functions import max

event.groupBy("Year", "Season").agg(max("Event").alias("MaxAthletes")).orderBy(desc("MaxAthletes")).show(1)

+----+------+--------------------+
|Year|Season|         MaxAthletes|
+----+------+--------------------+
|2004|Summer|Wrestling Women's...|
+----+------+--------------------+
only showing top 1 row



In [10]:
#Average age of male athletes in each country with respective game
from pyspark.sql.functions import avg

game_name = "Football"
df_filtered = event.filter((event.Sex == "M") & (event.Sport == game_name))

avg_age_by_country = df_filtered.groupBy("Team").agg(avg("Age").alias("Avg_Age"))

avg_age_by_country.show()

+-------------+------------------+
|         Team|           Avg_Age|
+-------------+------------------+
|       Russia|              21.3|
|     Paraguay|22.323529411764707|
|      Senegal|22.058823529411764|
|       Sweden|23.666666666666668|
|Upton Park FC|23.857142857142858|
|     Malaysia|24.733333333333334|
|         Fiji|            22.625|
|       Turkey| 24.83783783783784|
|         Iraq|23.661971830985916|
|      Germany|24.145454545454545|
|  Afghanistan|              null|
|        Sudan|              28.0|
|       France| 24.08955223880597|
|       Greece|             23.25|
|      Algeria|          23.09375|
|     Slovakia| 22.77777777777778|
|    Argentina| 22.53543307086614|
|      Belgium| 25.25862068965517|
|Great Britain|25.691666666666666|
|        Qatar|19.814814814814813|
+-------------+------------------+
only showing top 20 rows



In [11]:
#Average age of female athletes in each country with respective game

from pyspark.sql.functions import avg

game_name = "Speed Skating"
df_filtered = event.filter((event.Sex == "F") & (event.Sport == game_name))

avg_age_by_country = df_filtered.groupBy("Team").agg(avg("Age").alias("Avg_Age"))

avg_age_by_country.show()

+-------------+------------------+
|         Team|           Avg_Age|
+-------------+------------------+
|       Russia| 25.95049504950495|
|       Sweden| 22.26153846153846|
|      Germany| 26.68211920529801|
|United States|23.579646017699115|
|        China|23.074766355140188|
| Soviet Union|24.583333333333332|
|        Italy|25.515151515151516|
|       Norway| 23.93975903614458|
|      Ukraine|23.857142857142858|
| East Germany| 21.40983606557377|
|  North Korea|              20.5|
|  South Korea|19.677419354838708|
|  Switzerland|              24.4|
| Unified Team| 23.22222222222222|
|       Canada|24.048387096774192|
|        Japan|23.701421800947866|
|      Austria|25.058823529411764|
|   Kazakhstan|24.761904761904763|
|  Netherlands| 24.44776119402985|
|       France|20.956521739130434|
+-------------+------------------+
only showing top 20 rows



In [22]:
#which owns the most medals according to the respective game

game_name = "Tennis"
game_df = event.filter(event.Sport == game_name)

if game_df.count() == 0:
    print(f"No events found for {game_name}.")
else:
    medals_df = game_df.groupBy("Name").agg({"Medal": "count"})

    if medals_df.count() == 0:
        print(f"No medals found for {game_name}.")
    else:
        top_athlete = medals_df.orderBy("count(Medal)", ascending=False).first()["Name"]
        print(f"The athlete with the most medals in {game_name} is {top_athlete}.")

The athlete with the most medals in Tennis is Leander Adrian Paes.


In [23]:
#what is the average height and weight of female and male athletes in each sport
from pyspark.sql.functions import avg

athlete_df = event.filter((event.Sex == 'M') | (event.Sex == 'F'))

result_df = athlete_df.groupBy("Sport", "Sex").agg(avg("Height"), avg("Weight"))

result_df.show()

+----------------+---+------------------+------------------+
|           Sport|Sex|       avg(Height)|       avg(Weight)|
+----------------+---+------------------+------------------+
|       Athletics|  M| 179.7508488574837| 73.81517898651789|
|        Softball|  F|169.39508928571428| 67.47165532879819|
|          Tennis|  F|172.31921487603304| 62.08324661810614|
|        Canoeing|  M|181.20725995316158| 79.95920281359906|
|   Equestrianism|  F|167.88072601555749| 58.60469157254561|
|       Taekwondo|  F|170.81164383561645| 61.13682432432432|
|          Diving|  F| 161.1736040609137| 53.56656346749226|
|        Football|  F|167.68431771894095| 60.94087665647299|
|       Wrestling|  F|163.86513157894737|60.554455445544555|
|  Figure Skating|  M|176.08588957055215|  69.5840108401084|
|      Gymnastics|  M|167.63883336361985| 63.34486129212437|
|      Water Polo|  M|186.80713974749673| 87.71294223018361|
|Art Competitions|  F|             160.0|              null|
|   Equestrianism|  M|17

In [25]:
#what are the different sports and year held in Paris?

paris_df = event.filter(event.City == 'Paris')

sports = paris_df.select('Sport').distinct().collect()
years = paris_df.select('Year').distinct().collect()

print("Sports held in Paris:")
for sport in sports:
    print(sport.Sport)

print("\nYears in which Paris held the Olympics:")
for year in years:
    print(year.Year)


Sports held in Paris:
Gymnastics
Tennis
Boxing
Golf
Rowing
Sailing
Swimming
Equestrianism
Croquet
Athletics
Polo
Shooting
Diving
Wrestling
Water Polo
Weightlifting
Tug-Of-War
Cricket
Modern Pentathlon
Rugby
Cycling
Archery
Fencing
Football
Art Competitions
Basque Pelota

Years in which Paris held the Olympics:
1924
1900


In [31]:
#Which countries won the maximum gold Medals in the last held olympic competitions?

latest_year = event.select("Year").distinct().orderBy("Year", ascending=False).first().Year
latest_year_df = event.filter(event.Year == latest_year)
medals_df = year_df.groupBy("NOC", "Medal").count()
gold_df = medals_df.filter(medals_df.Medal == 'Gold')
gold_df = gold_df.orderBy('count', ascending=False)

print(f"Countries that won the most gold medals in the {year} Olympics:")
for row in gold_df.take(10):
    print(row.NOC, row['count'])

Countries that won the most gold medals in the 2016 Olympics:
USA 138
GBR 64
RUS 52
GER 49
CHN 46
BRA 36
AUS 23
ARG 21
FRA 20
JPN 17


In [33]:
#Which country has the highest medals (gold, silver, bronze)?

medals_df = event.groupBy("NOC", "Medal").count()
total_df = medals_df.groupBy("NOC").agg({"count": "sum"}).withColumnRenamed("sum(count)", "total")
total_df = total_df.orderBy('total', ascending=False)

print(f"The country with the highest number of medals is {total_df.first().NOC} with a total of {total_df.first().total} medals.")

The country with the highest number of medals is USA with a total of 18331 medals.
